Update thêm tham số `--limit` vào file `main_listVideosExcel_2_listTranscript.py`





Tôi sẽ thêm tham số `--limit` vào file `main_listVideosExcel_2_listTranscript.py`:

```python:Crawl_Youtube_rapidapi/idVideoYoutube2Transcription_RapidAPI/main_listVideosExcel_2_listTranscript.py
import logging
import argparse
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import List, Dict
import sys
from utils_readExcel import ExcelReader
from utils_saveExcel import TranscriptExcelSaver
from def_IDVideoYoutube2Transcript import YouTubeTranscriptFetcher

# Thiết lập logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)
logger = logging.getLogger(__name__)

def process_video(video: Dict, transcript_fetcher: YouTubeTranscriptFetcher, 
                 transcript_saver: TranscriptExcelSaver) -> Dict:
    """
    Xử lý một video: lấy transcript và lưu vào Excel.
    """
    try:
        video_id = video['videoId']
        logger.info(f"Đang xử lý video: {video['title']} ({video_id})")

        # Lấy transcript
        transcript_data = transcript_fetcher.get_transcript(video_id)
        if not transcript_data:
            return {
                'video_id': video_id,
                'status': 'error',
                'message': 'Không thể lấy transcript'
            }

        # Lưu transcript vào Excel
        excel_path = transcript_saver.save_transcript(transcript_data, video)
        if not excel_path:
            return {
                'video_id': video_id,
                'status': 'error',
                'message': 'Không thể lưu transcript vào Excel'
            }

        return {
            'video_id': video_id,
            'status': 'success',
            'excel_path': excel_path
        }

    except Exception as e:
        logger.exception(f"Lỗi khi xử lý video {video_id}: {str(e)}")
        return {
            'video_id': video_id,
            'status': 'error',
            'message': str(e)
        }

def main():
    parser = argparse.ArgumentParser(description='Lấy transcript cho danh sách video từ file Excel')
    parser.add_argument('--input', required=True, help='Đường dẫn đến file Excel chứa danh sách video')
    parser.add_argument('--max-workers', type=int, default=5, help='Số luồng xử lý tối đa')
    parser.add_argument('--limit', type=int, help='Số lượng video muốn lấy (mặc định: tất cả)')  # Thêm tham số limit
    args = parser.parse_args()

    # Khởi tạo các đối tượng
    excel_reader = ExcelReader()
    transcript_fetcher = YouTubeTranscriptFetcher()
    transcript_saver = TranscriptExcelSaver()

    # Đọc danh sách video từ Excel
    videos = excel_reader.read_videos_from_excel(args.input)
    if not videos:
        logger.error("Không thể đọc danh sách video từ file Excel")
        sys.exit(1)

    # Giới hạn số lượng video nếu có tham số limit
    if args.limit and args.limit > 0:
        videos = videos[:args.limit]
        logger.info(f"Giới hạn xử lý {args.limit} video đầu tiên")

    # Xử lý song song các video
    results = []
    with ThreadPoolExecutor(max_workers=args.max_workers) as executor:
        future_to_video = {
            executor.submit(process_video, video, transcript_fetcher, transcript_saver): video 
            for video in videos
        }
        
        for future in as_completed(future_to_video):
            video = future_to_video[future]
            try:
                result = future.result()
                results.append(result)
                logger.info(f"Kết quả xử lý video {video['videoId']}: {result['status']}")
            except Exception as e:
                logger.exception(f"Lỗi khi xử lý video {video['videoId']}: {str(e)}")

    # In tổng kết
    success_count = sum(1 for r in results if r['status'] == 'success')
    logger.info(f"Đã xử lý xong {len(results)} video, thành công: {success_count}")

if __name__ == "__main__":
    main()
```

Những thay đổi chính:

1. Thêm tham số `--limit`:
```python
parser.add_argument('--limit', type=int, help='Số lượng video muốn lấy (mặc định: tất cả)')
```

2. Thêm logic xử lý limit:
```python
if args.limit and args.limit > 0:
    videos = videos[:args.limit]
    logger.info(f"Giới hạn xử lý {args.limit} video đầu tiên")
```

Bây giờ bạn có thể chạy script với tham số `--limit` để giới hạn số lượng video muốn xử lý:

```bash
# Xử lý 5 video đầu tiên
python main_listVideosExcel_2_listTranscript.py --input listMoxieVideosYoutube.xlsx --limit 5

# Xử lý 10 video với 3 luồng
python main_listVideosExcel_2_listTranscript.py --input listMoxieVideosYoutube.xlsx --limit 10 --max-workers 3

# Xử lý tất cả video (không dùng --limit)
python main_listVideosExcel_2_listTranscript.py --input listMoxieVideosYoutube.xlsx
```
